In [ ]:
num_splits=20 #How many sections to split the data up into
num_bs=100 #How many bootstrap iterations to do
num_amts=10

R2_all, R2N_all = [], []
for o in range(3):
    if o==0:
        DF = df0
        yTest_all = yTest_all0
        yTestPred_all = yTestPred_all0
        yTestNPred_all = yTestNPred_all0
        yTestN_all = yTestN_all0
    elif o==1:
        DF = df1
        yTest_all = yTest_all1
        yTestPred_all = yTestPred_all1
        yTestNPred_all = yTestNPred_all1
        yTestN_all = yTestN_all1
    else:
        DF = df2
        yTest_all = yTest_all2
        yTestPred_all = yTestPred_all2
        yTestNPred_all = yTestNPred_all2
        yTestN_all = yTestN_all2
        
    tru = [yTest_all[i] for i in DF.index[DF['model']==0]]
    
    R2_bs_all, R2_bsN_all = [], []
    for m in range(8):
        wf = [yTestPred_all[i] for i in DF.index[DF['model']==m]]
        wf_null_pred = [yTestNPred_all[i] for i in DF.index[DF['model']==m]]
        wf_null = [yTestN_all[i] for i in DF.index[DF['model']==m]]

        R2_bs = np.empty([num_bs,num_amts,2])
        R2_bs_null = np.empty([num_bs,num_amts,2])

        for n in range(2):
            for d_amt in range(num_amts):
                num_test_examples = tru[d_amt].shape[0]
                num_examples_per_split = int(np.round(np.divide(num_test_examples,num_splits)))

                if n==0:
                    y_test_temp = tru[d_amt]
                    y_pred_temp = wf[d_amt]
                elif n==1:
                    y_test_temp = wf_null[d_amt]
                    y_pred_temp = wf_null_pred[d_amt]

                y_test_bs = np.empty([num_splits,num_examples_per_split,tru[0].shape[1]])
                y_pred_bs = np.empty([num_splits,num_examples_per_split,tru[0].shape[1]])

                for i in range(num_splits): 
                    idx=np.arange(num_examples_per_split*i,num_examples_per_split*(i+1)) #These are the indices of the data points from the given split        

                    y_test_bs[i,:,:]=y_test_temp[idx,:]    
                    y_pred_bs[i,:,:]=y_pred_temp[idx,:]

                for i in range(num_bs): #Loop through bootstrap iterations   
                    #Step 1
                    random_idxs=np.floor(num_splits*np.random.rand(num_splits)).astype(int)

                    #Step 2
                    y_test = np.reshape(y_test_bs[random_idxs,:,:],[y_test_bs.shape[0]*y_test_bs.shape[1],y_test_bs.shape[2]])    
                    y_pred = np.reshape(y_pred_bs[random_idxs,:,:],[y_test_bs.shape[0]*y_test_bs.shape[1],y_test_bs.shape[2]]) 

                    #Step 3
                    if n==0:
                        R2_bs[i,d_amt,:] = get_R2(y_test,y_pred)
                    elif n==1:
                        R2_bs_null[i,d_amt,:] = get_R2(y_test,y_pred)

        R2_bs_all.append(R2_bs.reshape(R2_bs.shape[0]*R2_bs.shape[1], R2_bs.shape[2]))
        R2_bsN_all.append(R2_bs_null.reshape(R2_bs_null.shape[0]*R2_bs_null.shape[1], R2_bs_null.shape[2]))
    R2_all.append(R2_bs_all)
    R2N_all.append(R2_bsN_all)